In [216]:
import tensorflow;
import os;

os.environ["TL_BACKEND"] = "tensorflow";

import tensorlayerx;
import numpy;

from tensorflow.keras.datasets import mnist;

In [217]:
print(tensorflow.config.list_physical_devices('GPU'));

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [218]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

# Split the test and val by 50:50
test_val_images_split = numpy.array_split(test_images, 2);
test_val_labels_split = numpy.array_split(test_labels, 2);

test_images = test_val_images_split[0];
test_labels = test_val_labels_split[0];

val_images = test_val_images_split[1];
val_labels = test_val_labels_split[1];

In [219]:
# Recall step 1
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255;
test_images = test_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;
val_images = val_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;

# Recall step 2
train_labels = tensorflow.one_hot(train_labels, depth = 10);
test_labels = tensorflow.one_hot(test_labels, depth = 10);
val_labels = tensorflow.one_hot(val_labels, depth = 10);

In [220]:
from tensorlayerx.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear;
from tensorlayerx.model import TrainOneStep;
from tensorlayerx.optimizers import Adam;
from tensorlayerx.losses import cross_entropy_seq;
from tqdm import tqdm;

In [221]:
class CNNModel:
    def __init__(self):
        self.conv1 = Conv2d(out_channels = 32, kernel_size = (3, 3), act = "relu", in_channels = 1, name = "conv1");
        self.pool1 = MaxPool2d(kernel_size = (2, 2), name = "pool1");

        self.conv2 = Conv2d(out_channels = 32, kernel_size = (3, 3), act = "relu", name = "conv2");
        self.pool2 = MaxPool2d(kernel_size = (2, 2), name = "pool2");

        self.flat = Flatten(name = "flat");

        self.dense = Linear(out_features = 64, act = "relu", name = "dense");

        self.output = Linear(out_features = 3, act = "relu", name = "output");
    
    def compile(self):
        model = Sequential([
            self.conv1,
            self.pool1,
            self.conv2,
            self.pool2,
            self.flat,
            self.dense,
            self.output
        ]);

        print("------------------------------------------------")
        print("               MNIST Tensorlayerx               ");
        print("------------------------------------------------")
        print("Neuron: ", sum(layer.out_features for layer in model if hasattr(layer, 'out_features')))
        print(model);

        return model;

In [222]:
class ManualTrainer:

    """
    ManualTrainer

    This class is mimicing the Tensorflow's automatic fit. 
    However, since the adversarial training algorithm may requires TrainOneStep.

    @params
        epoch: number of epoch
        batch_size: total data per training minibatch (For every epoch elapsed, train the data with minibatches.)
        model: model to be trained (tensorlayerx.nn.Sequential)
        optimizer: optimizer algorithm to be used for training (tensorlayerx.optimizers)
        loss: loss function to be used for training (tensorlayerx.losses)
        
        ----------------------------------------------------------------
        Every parameters below are defined as array length of 2:
        train_set -> [ [X_set], [Y_set] ]
        val_set -> [ [X_set], [Y_set] ]
    """
    def __init__(self, epoch: int, batch_size: int, model, optimizer, loss, train_set, val_set = []):
        self.epoch = epoch;
        self.batch_size = batch_size;
        self.optimizer = optimizer;
        self.loss = loss;
        self.model = model;
        
        # Dataset
        self.train_set = [
            [train_set[0]],
            [train_set[1]],
        ];

        # If the validation is not set, then the validation set will be using training set instead
        self.val_set = [
            [train_set[0]] if (len(val_set) == 0) else [val_set[0]],
            [train_set[1]] if (len(val_set) == 0) else [val_set[1]],
        ];


        self.average_training_loss = 0.0;
        self.validation_loss = 0.0;
        self.validation_accuracy = 0.0;

        # Train One Step
    
    def fitting(self):
        loss = 0.0;

        feed_forward = TrainOneStep(self.model, optimizer = self.optimizer, train_weights = True);

        # Start of epoch
        for epoch in tqdm(range(self.epoch)):
            epoch_loss = 0.0;

            # Start of batch training
            for batch in range(self.batch_size):
                start_dataset_index = batch * self.batch_size;
                end_dataset_index = (batch + 1) * self.batch_size;

                feature_set = self.train_set[0][start_dataset_index:end_dataset_index];
                label_set = self.train_set[1][start_dataset_index:end_dataset_index];

                training_loss_value = feed_forward(feature_set, label_set);

                epoch_loss += training_loss_value;
            # End of batch training
            
            self.average_training_loss = epoch_loss / self.batch_size;

            # The loss now count with validation set 
            loss, accuracy = model.evaluate(self.val_set[0], self.val_set[1]);

            self.validation_loss = loss;
            self.validation_accuracy = accuracy;

            print(f"Epoch {epoch + 1}/{self.epoch} | loss: {self.validation_loss} | accuracy: {self.validation_accuracy}")

        # End of epoch
        return self.model;

In [223]:
cnn = CNNModel();
model = cnn.compile();

trainer = ManualTrainer(
    epoch = 5, 
    batch_size = 64, 
    optimizer = Adam(lr = 1e-3),
    loss = cross_entropy_seq,
    model = model,
    train_set = [train_images, train_labels],
    val_set = [val_images, val_labels]
);

model = trainer.fitting();
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

[TLX] Conv2d conv1: out_channels : 32 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
[TLX] MaxPool2d pool1: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
[TLX] Conv2d conv2: out_channels : 32 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
[TLX] MaxPool2d pool2: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
[TLX] Flatten flat:
[TLX] Linear  dense: 64 ReLU
[TLX] Linear  output: 3 ReLU
------------------------------------------------
               MNIST Tensorlayerx               
------------------------------------------------
Neuron:  67
Sequential<
  (0): Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=SAME, ReLU, name='conv1')
  (1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=SAME, return_mask=False, name='pool1')
  (2): Conv2d(in_channels=None, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=SAME, ReLU, name='conv2')
  (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2),

  0%|                                                                                                                                                          | 0/5 [00:00<?, ?it/s]


TypeError: TrainOneStep.__call__() missing 1 required positional argument: 'label'